# Sara Gets a Diamond Case

# 1. Getting Ready - Import pacakages

Import packages we will use:

1. Numpy. This is a de-facto standard library for linear algebra in Python. Info: https://numpy.org/doc/
2. Pandas. It is most commonly used library for data engineering. Info: https://pandas.pydata.org
3. Statsmodels. Commonly used for basic statistical analysis. Info: https://www.statsmodels.org/stable/index.html.
4. Plotly express. Info: https://plotly.com/python/plotly-express/


In [50]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import plotly.express as px
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 2. Data Engineering

### Step 2.1: Load and explore the data

Upload the data file "UV6248-XLS-ENG.xls" and read the provided datafile into a dataframe  using **pd.read_excel()** function.

Rename the Carat Weight variable to get rid of the space in the name.

Remove records with null values.

In [51]:
df = pd.read_excel("UV6248-XLS-ENG.xls", sheet_name = "Raw Data", skiprows=2)
df = df.rename(columns={'Carat Weight': 'CaratWeight'})
df = df.dropna()

Display the first 7 rows of data using the **head(7)** command.

In [52]:
df.head(7)
#You can change how many rows of data you want to see by adding a parameter to the head(rows) function: e.g

,ID,CaratWeight,Cut,Color,Clarity,Polish,Symmetry,Report,Price
0,1,1.10,Ideal,H,SI1,VG,EX,GIA,5169.0
1,2,0.83,Ideal,H,VS1,ID,ID,AGSL,3470.0
2,3,0.85,Ideal,H,SI1,EX,EX,GIA,3183.0
3,4,0.91,Ideal,E,SI1,VG,VG,GIA,4370.0
4,5,0.83,Ideal,G,SI1,EX,EX,GIA,3171.0
5,6,1.53,Ideal,E,SI1,ID,ID,AGSL,12791.0
6,7,1.00,Very Good,D,SI1,VG,G,GIA,5747.0


Print out the count, mean, and standard deviation for all numerical variables formatted to two decimal points using **.describe()** command.

In [53]:
df.describe().loc[['count', 'mean', 'std']].round(2)
#Note that you only get summary stats for the numerical fields, not categories

,ID,CaratWeight,Price
count,6000.0,6000.00,6000.00
mean,3000.5,1.33,11791.58
std,1732.2,0.48,10184.35


Check which variables are categorical by listing the data type for all variables using **.dtypes**:

In [54]:
df.dtypes

ID               int64
CaratWeight    float64
Cut             object
Color           object
Clarity         object
Polish          object
Symmetry        object
Report          object
Price          float64
dtype: object

If you want to look up the values for each categorical variable, you can use **.unique()**:

There, are, of course, many other ways to get the list of unique categorical variables. Here is another option:

In [55]:
#Print a list of categorical variables
categorical_variables = df.select_dtypes(include=['object']).columns.tolist()
print("Categorical Variables:", categorical_variables)

Categorical Variables: ['Cut', 'Color', 'Clarity', 'Polish', 'Symmetry', 'Report']


Print out the number of times you can find the value "FL" for "Clarity"

In [56]:
FL_count = (df['Clarity'] == 'FL').sum()

# Print the result
print("Count of 'FL' in Clarity:", FL_count)

Count of 'FL' in Clarity: 4


This is bad news :(

  You do not want to include a variable for explanation where one category has 4 records in it, and another - 1000.


In [57]:
df['Clarity'] = df['Clarity'].replace({'IF': 'IF_FL', 'FL': 'IF_FL'}) #Since 'FL' has only 4 records, we combine it with another category "FL"
print(df['Clarity'].value_counts())

SI1      2059
VS2      1575
VS1      1192
VVS2      666
VVS1      285
IF_FL     223
Name: Clarity, dtype: int64


## Step 2.2 Visual Exploration of the Data


Explore impact of different categorical variables on our data.

In [58]:
fig = px.scatter(df,
                 x="CaratWeight",
                 y="Price",
                 color = 'Color',
                 height = 350
                )
fig.show()


Let's explore whether taking a log of X or Y helps the relationship.

In [59]:
from math import log
df_log = df.copy()
df_log['Price']=df_log['Price'].transform(log)
df_log['CaratWeight']=df_log['CaratWeight'].transform(log)
print(df_log['Price'].head(10))

fig = px.scatter(df_log,
                 x="CaratWeight",
                 y="Price",
                 color = 'Color',
                 height = 350,
                 labels={'Price':'Log of Price'#, 'CaratWeight': 'Log of CaratWeight'
                         }
                )
fig.show()

0    8.550435
1    8.151910
2    8.065579
3    8.382518
4    8.061802
5    9.456497
6    8.656433
7    9.254357
8    9.831401
9    8.944550
Name: Price, dtype: float64


## Step 2.3 Create dummy variables for categorical variables using **pd.get_dummies()**:

In [60]:
df = pd.get_dummies(data=df, columns = ['Color', 'Report'], drop_first=True)
df.head() # Check how the data looks now
# ['Color','Clarity','Cut','Polish','Symmetry','Report']

,ID,CaratWeight,Cut,Clarity,Polish,Symmetry,Price,Color_E,Color_F,Color_G,Color_H,Color_I,Report_GIA
0,1,1.10,Ideal,SI1,VG,EX,5169.0,0,0,0,1,0,1
1,2,0.83,Ideal,VS1,ID,ID,3470.0,0,0,0,1,0,0
2,3,0.85,Ideal,SI1,EX,EX,3183.0,0,0,0,1,0,1
3,4,0.91,Ideal,SI1,VG,VG,4370.0,1,0,0,0,0,1
4,5,0.83,Ideal,SI1,EX,EX,3171.0,0,0,1,0,0,1


## Step 2.4 Check for multicollinearity

Look at the correlation matrix to see if we have potential multicollinearity

Encode ordinal categorical variables using an increasing scale

In [61]:
scale_map_cut = {'Fair':1, 'Good':2, 'Very Good':3, 'Ideal':4,  'Signature-Ideal':5}
scale_map_clarity = {'SI1':1, 'VVS2':3, 'VVS1':3, 'VS2':2, 'VS1':2, 'IF_FL':4}#'IF':6, 'FL':6}
scale_map_polish = {'G':1, 'VG':2, 'EX':3, 'ID':3}

df["Cut"] = df["Cut"].replace(scale_map_cut)
df["Clarity"] = df["Clarity"].replace(scale_map_clarity)
df["Polish"] = df["Polish"].replace(scale_map_polish)
df["Symmetry"] = df["Symmetry"].replace(scale_map_polish)
df.head()

,ID,CaratWeight,Cut,Clarity,Polish,Symmetry,Price,Color_E,Color_F,Color_G,Color_H,Color_I,Report_GIA
0,1,1.10,4,1,2,3,5169.0,0,0,0,1,0,1
1,2,0.83,4,2,3,3,3470.0,0,0,0,1,0,0
2,3,0.85,4,1,3,3,3183.0,0,0,0,1,0,1
3,4,0.91,4,1,2,2,4370.0,1,0,0,0,0,1
4,5,0.83,4,1,3,3,3171.0,0,0,1,0,0,1


In [62]:
print(df.dtypes)
df.corr().style.background_gradient(cmap='RdBu_r', axis=None)
df.drop(['Price','ID'], axis=1).corr().style.background_gradient(cmap='coolwarm', axis=None)
#"axis=None" option above indicates that the colors are assigned based on the values in the whole matrix
# Other good color maps: 'RdBu_r' & 'PuOr_r' & 'coolwarm'

ID               int64
CaratWeight    float64
Cut              int64
Clarity          int64
Polish           int64
Symmetry         int64
Price          float64
Color_E          uint8
Color_F          uint8
Color_G          uint8
Color_H          uint8
Color_I          uint8
Report_GIA       uint8
dtype: object


,CaratWeight,Cut,Clarity,Polish,Symmetry,Color_E,Color_F,Color_G,Color_H,Color_I,Report_GIA
CaratWeight,1.000000,0.072943,0.102424,0.068080,0.054716,-0.089155,-0.024898,0.037891,0.045767,0.058235,0.011461
Cut,0.072943,1.000000,0.166885,0.398449,0.520504,-0.053341,-0.045228,0.057314,0.004432,0.012034,-0.276354
Clarity,0.102424,0.166885,1.000000,0.125463,0.119310,-0.054752,0.014908,0.129393,-0.052125,-0.064204,-0.080853
Polish,0.068080,0.398449,0.125463,1.000000,0.593765,-0.033634,-0.026723,0.062853,0.005671,-0.006967,-0.285906
Symmetry,0.054716,0.520504,0.119310,0.593765,1.000000,-0.059661,-0.017800,0.054295,0.013411,0.006019,-0.304807
Color_E,-0.089155,-0.053341,-0.054752,-0.033634,-0.059661,1.000000,-0.173963,-0.222948,-0.180740,-0.169293,0.059315
Color_F,-0.024898,-0.045228,0.014908,-0.026723,-0.017800,-0.173963,1.000000,-0.260326,-0.211042,-0.197675,0.028410
Color_G,0.037891,0.057314,0.129393,0.062853,0.054295,-0.222948,-0.260326,1.000000,-0.270468,-0.253338,-0.021583
Color_H,0.045767,0.004432,-0.052125,0.005671,0.013411,-0.180740,-0.211042,-0.270468,1.000000,-0.205377,-0.029140
Color_I,0.058235,0.012034,-0.064204,-0.006967,0.006019,-0.169293,-0.197675,-0.253338,-0.205377,1.000000,-0.072709


Compute Variance Inflation Factors (VIF)

In [63]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
# VIF dataframe

vif_data = pd.DataFrame()
vif_data["feature"] = df.drop(['Price','ID'], axis=1).columns


# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(df.drop(['Price','ID'], axis=1).values, i)
                          for i in range(len(df.drop(['Price','ID'], axis=1).columns))]
print(vif_data)

        feature        VIF
0   CaratWeight   8.601107
1           Cut  21.700758
2       Clarity   6.721845
3        Polish  21.250068
4      Symmetry  20.539357
5       Color_E   1.954304
6       Color_F   2.269801
7       Color_G   3.003294
8       Color_H   2.374629
9       Color_I   2.219651
10   Report_GIA   6.055876


## Step 2.5: Split the data into X and Y.
The vector of Y ("dependent") variable should contain the Price.
The matrix of X ("independent") variables should contain everything we will use to predict Y


In [64]:
Y = df[(['Price'])]
Y.head() # it's always a good idea to peak at your output

,Price
0,5169.0
1,3470.0
2,3183.0
3,4370.0
4,3171.0


In [65]:
X = df.drop(['Price','ID', 'Report_GIA'], axis=1)
#X = df[['CaratWeight']]
X.dtypes

CaratWeight    float64
Cut              int64
Clarity          int64
Polish           int64
Symmetry         int64
Color_E          uint8
Color_F          uint8
Color_G          uint8
Color_H          uint8
Color_I          uint8
dtype: object

# 3. Build regression models

## 3.1 Try a linear model

In [66]:
# In this package, by default, the regression will have no intercept, hence we need to manually add it to the X matrix, and call the result X_const
X_const = sm.add_constant(X)

# Fit a linear regression model with vector Y as dependent and matrix X_sm as independent
lm = sm.OLS(Y, X_const).fit()

# Display the summary of model results
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.852
Model:                            OLS   Adj. R-squared:                  0.852
Method:                 Least Squares   F-statistic:                     3460.
Date:                Mon, 05 Feb 2024   Prob (F-statistic):               0.00
Time:                        20:41:46   Log-Likelihood:                -58144.
No. Observations:                6000   AIC:                         1.163e+05
Df Residuals:                    5989   BIC:                         1.164e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -1.508e+04    312.362    -48.272      

### Plot the residuals

In [67]:
# Compute the residuals
results = pd.DataFrame()
results['Price'] = df['Price']
results['prediction_lm'] = lm.fittedvalues
results['residual_lm'] = lm.resid

fig = px.scatter(
    results, x='prediction_lm', y='residual_lm', height = 350,
    labels={'prediction_lm':'Predicted values using the Linear Model', 'residual_lm':'Residuals using the Linear Model'}
)
fig.show()

## 3.2 Try a log-linear model to the case data

In [68]:
from math import exp, log
Y_log = Y['Price'].transform(log)
log_linear_model = sm.OLS(Y_log, X_const).fit()
print(log_linear_model.summary())

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.957
Model:                            OLS   Adj. R-squared:                  0.957
Method:                 Least Squares   F-statistic:                 1.329e+04
Date:                Mon, 05 Feb 2024   Prob (F-statistic):               0.00
Time:                        20:41:47   Log-Likelihood:                 2969.4
No. Observations:                6000   AIC:                            -5917.
Df Residuals:                    5989   BIC:                            -5843.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           6.8830      0.012    584.337      

In [69]:
# Compute residuals
results['prediction_llm'] = log_linear_model.fittedvalues
results['residual_llm'] = log_linear_model.resid

fig = px.scatter(
    results, x='prediction_llm', y='residual_llm', height = 350,
    labels={'prediction_llm':'Predicted values using the Log-Linear Model', 'residual_llm':'Residuals'}
)
fig.show()


## 3.3 Fit a log-log model to the case data

In [70]:
#INSERT CODE HERE:
Y_log = Y['Price'].transform(log)
X_log = X.copy()
X_log['CaratWeight'] = X_log['CaratWeight'].transform(log)
X_log_const = sm.add_constant(X_log)
loglog = sm.OLS(Y_log, X_log_const).fit()
print(loglog.summary())


                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.977
Model:                            OLS   Adj. R-squared:                  0.977
Method:                 Least Squares   F-statistic:                 2.532e+04
Date:                Mon, 05 Feb 2024   Prob (F-statistic):               0.00
Time:                        20:41:47   Log-Likelihood:                 4840.8
No. Observations:                6000   AIC:                            -9660.
Df Residuals:                    5989   BIC:                            -9586.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           8.2891      0.008   1027.948      

In [71]:
# Compute residuals
results['prediction_ll'] = loglog.fittedvalues
results['residual_ll'] = loglog.resid

fig = px.scatter(
    results, x='prediction_ll', y='residual_ll', height = 350,
    labels={'prediction_ll':'Predicted values using the Log-Log Model', 'residual_ll':'Residuals'}
)
fig.show()

# 4. Cross-validation

The main machine learning principle that allows to answer this question -- cross-validation: splitting the data into training (80%) and testing (20%) subsets, training on the former and testing on the latter.

In [72]:
# Redefine X and Y for the training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

#Add a constant to the X's:
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

X_train_const.head()

,const,CaratWeight,Cut,Clarity,Polish,Symmetry,Color_E,Color_F,Color_G,Color_H,Color_I
3897,1.0,1.34,3,2,2,1,0,0,0,1,0
5628,1.0,2.57,4,2,3,2,0,0,0,1,0
1756,1.0,1.01,4,4,3,3,0,0,0,0,0
2346,1.0,2.09,4,2,2,2,0,1,0,0,0
2996,1.0,0.90,3,1,2,2,0,0,0,0,1


## 4.1 Check the Linear Model

In [73]:
# Fit a linear regression model to the training data
lm = sm.OLS(Y_train, X_train_const).fit()

# Use the trained model to predict the prices for the testing data. Call the vector of predicted prices Y_pred
Y_pred = lm.predict(X_test_const)
percent_errors = np.abs((Y_test['Price'] - Y_pred) / Y_test['Price']) *100
print("Linear Model MAPE = ", np.mean(percent_errors), "%")

Linear Model MAPE =  29.0981401074879 %


## 4.2 Check the Log-Linear Model


In [74]:
# Fit a log-linear regression model to the training data
Y_train_log = Y_train['Price'].transform(log)
llm = sm.OLS(Y_train_log, X_train_const).fit()

Y_pred_llm = np.exp(llm.predict(X_test_const))
percent_errors = np.abs((Y_test['Price'] - Y_pred_llm) / Y_test['Price']) *100
print("Log-Linear Model MAPE = ", np.mean(percent_errors), "%")

Log-Linear Model MAPE =  11.353567504847339 %


## 4.3 Check the Log-Log Model

In [75]:
# Fit a log-log regression model to the training data
X_train_log = X_train.copy()
X_train_log['CaratWeight'] = X_train_log['CaratWeight'].transform(log)
X_train_log_const = sm.add_constant(X_train_log)

X_test_log = X_test.copy()
X_test_log['CaratWeight'] = X_test_log['CaratWeight'].transform(log)
X_test_log_const = sm.add_constant(X_test_log)


loglog = sm.OLS(Y_train_log, X_train_log_const).fit()
print(loglog.summary())

Y_pred_loglog = np.exp(loglog.predict(X_test_log_const))
percent_errors = np.abs((Y_test['Price'] - Y_pred_loglog) / Y_test['Price']) *100
print("Log-Log Model MAPE = ", np.mean(percent_errors), "%")

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.977
Model:                            OLS   Adj. R-squared:                  0.977
Method:                 Least Squares   F-statistic:                 2.018e+04
Date:                Mon, 05 Feb 2024   Prob (F-statistic):               0.00
Time:                        20:41:48   Log-Likelihood:                 3848.1
No. Observations:                4800   AIC:                            -7674.
Df Residuals:                    4789   BIC:                            -7603.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           8.2872      0.009    917.586      

# 5. How to improve the model further? Push the model further with your team

Recall our visualizations, we observed multiple effects:

1.   Price increases with Carat Weight -- our model already "learned" that -- Carat Weight is one of the variables, and its coefficient is positive
2.   Price increases exponentially with Carat Weight -- our model already "learned" that with log-log transformation.
3.   Diamonds with "better" colors are more expensive -- our model already "somewhat learned" that too: the best ("D") color diamond is in the intercept, and the other Color coefficients are negative. However, our model does not let the slope of the line change based on the color. What can you do? Perhaps, try some interactions.
4.   There seems to be a disconitnuity around 2 carats, maybe even one more around 1 carat. Try splitting the dataset into multiple models, and fit a different model in each interval.



##Model 1 interaction with log (the final chosen model)

In [76]:
X_log_1 = X_log.copy()
X_log_1.dtypes

CaratWeight    float64
Cut              int64
Clarity          int64
Polish           int64
Symmetry         int64
Color_E          uint8
Color_F          uint8
Color_G          uint8
Color_H          uint8
Color_I          uint8
dtype: object

In [77]:
X_log_1['inter_E'] = X_log_1['Color_E'] * X_log_1['CaratWeight']
X_log_1['inter_F'] = X_log_1['Color_F'] * X_log_1['CaratWeight']
X_log_1['inter_G'] = X_log_1['Color_G'] * X_log_1['CaratWeight']
X_log_1['inter_H'] = X_log_1['Color_H'] * X_log_1['CaratWeight']
X_log_1['inter_I'] = X_log_1['Color_I'] * X_log_1['CaratWeight']

X_log_1.head()

,CaratWeight,Cut,Clarity,Polish,Symmetry,Color_E,Color_F,Color_G,Color_H,Color_I,inter_E,inter_F,inter_G,inter_H,inter_I
0,0.095310,4,1,2,3,0,0,0,1,0,0.000000,0.0,0.00000,0.095310,0.0
1,-0.186330,4,2,3,3,0,0,0,1,0,-0.000000,-0.0,-0.00000,-0.186330,-0.0
2,-0.162519,4,1,3,3,0,0,0,1,0,-0.000000,-0.0,-0.00000,-0.162519,-0.0
3,-0.094311,4,1,2,2,1,0,0,0,0,-0.094311,-0.0,-0.00000,-0.000000,-0.0
4,-0.186330,4,1,3,3,0,0,1,0,0,-0.000000,-0.0,-0.18633,-0.000000,-0.0


In [78]:
#MODEL1 SUMMARY
X_log_const_1 = sm.add_constant(X_log_1)
loglog_1 = sm.OLS(Y_log, X_log_const_1).fit()
print(loglog_1.summary())


                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.979
Model:                            OLS   Adj. R-squared:                  0.979
Method:                 Least Squares   F-statistic:                 1.855e+04
Date:                Mon, 05 Feb 2024   Prob (F-statistic):               0.00
Time:                        20:41:48   Log-Likelihood:                 5120.5
No. Observations:                6000   AIC:                        -1.021e+04
Df Residuals:                    5984   BIC:                        -1.010e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           8.2644      0.008   1047.483      

In [79]:
results['prediction_ll_1'] = loglog_1.fittedvalues
results['residual_ll_1'] = loglog_1.resid

fig = px.scatter(
    results, x='prediction_ll_1', y='residual_ll_1', height = 350,
    labels={'prediction_ll_1':'Predicted values using the Log-Log Model and Interaction of color', 'residual_ll_1':'Residuals'}
)
fig.show()

In [80]:

X_train_log_1 = X_train_log.copy()
X_train_log_1['inter_E'] = X_train_log_1['Color_E'] * X_train_log_1['CaratWeight']
X_train_log_1['inter_F'] = X_train_log_1['Color_F'] * X_train_log_1['CaratWeight']
X_train_log_1['inter_G'] = X_train_log_1['Color_G'] * X_train_log_1['CaratWeight']
X_train_log_1['inter_H'] = X_train_log_1['Color_H'] * X_train_log_1['CaratWeight']
X_train_log_1['inter_I'] = X_train_log_1['Color_I'] * X_train_log_1['CaratWeight']
X_train_log_const_1 = sm.add_constant(X_train_log_1)

X_test_log_1 = X_test_log.copy()

X_test_log_1['inter_E'] = X_test_log_1['Color_E'] * X_test_log_1['CaratWeight']
X_test_log_1['inter_F'] = X_test_log_1['Color_F'] * X_test_log_1['CaratWeight']
X_test_log_1['inter_G'] = X_test_log_1['Color_G'] * X_test_log_1['CaratWeight']
X_test_log_1['inter_H'] = X_test_log_1['Color_H'] * X_test_log_1['CaratWeight']
X_test_log_1['inter_I'] = X_test_log_1['Color_I'] * X_test_log_1['CaratWeight']

X_test_log_const_1 = sm.add_constant(X_test_log_1)

loglog_1 = sm.OLS(Y_train_log, X_train_log_const_1).fit()
print(loglog_1.summary())
# print(Y_pred_loglog_1.head(7))
Y_pred_loglog_1 = np.exp(loglog_1.predict(X_test_log_const_1))
percent_errors_1 = np.abs((Y_test['Price'] - Y_pred_loglog_1) / Y_test['Price']) *100
print("Log-Log Model 1 MAPE = ", np.mean(percent_errors_1), "%")

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.979
Model:                            OLS   Adj. R-squared:                  0.979
Method:                 Least Squares   F-statistic:                 1.485e+04
Date:                Mon, 05 Feb 2024   Prob (F-statistic):               0.00
Time:                        20:41:49   Log-Likelihood:                 4082.2
No. Observations:                4800   AIC:                            -8132.
Df Residuals:                    4784   BIC:                            -8029.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           8.2631      0.009    938.402      

##Model 2 interaction with original

In [81]:
#Model 2 interaction with original

X_log_2 = X.copy()
X_log_2.dtypes

CaratWeight    float64
Cut              int64
Clarity          int64
Polish           int64
Symmetry         int64
Color_E          uint8
Color_F          uint8
Color_G          uint8
Color_H          uint8
Color_I          uint8
dtype: object

In [82]:
X_log_2['inter_E'] = X_log_2['Color_E'] * X_log_2['CaratWeight']
X_log_2['inter_F'] = X_log_2['Color_F'] * X_log_2['CaratWeight']
X_log_2['inter_G'] = X_log_2['Color_G'] * X_log_2['CaratWeight']
X_log_2['inter_H'] = X_log_2['Color_H'] * X_log_2['CaratWeight']
X_log_2['inter_I'] = X_log_2['Color_I'] * X_log_2['CaratWeight']

X_log_2['CaratWeight'] = X_log_2['CaratWeight'].transform(log)
X_log_2.head()

,CaratWeight,Cut,Clarity,Polish,Symmetry,Color_E,Color_F,Color_G,Color_H,Color_I,inter_E,inter_F,inter_G,inter_H,inter_I
0,0.095310,4,1,2,3,0,0,0,1,0,0.00,0.0,0.00,1.10,0.0
1,-0.186330,4,2,3,3,0,0,0,1,0,0.00,0.0,0.00,0.83,0.0
2,-0.162519,4,1,3,3,0,0,0,1,0,0.00,0.0,0.00,0.85,0.0
3,-0.094311,4,1,2,2,1,0,0,0,0,0.91,0.0,0.00,0.00,0.0
4,-0.186330,4,1,3,3,0,0,1,0,0,0.00,0.0,0.83,0.00,0.0


In [83]:
#MODEL2 SUMMARY
X_log_const_2 = sm.add_constant(X_log_2)
loglog_2 = sm.OLS(Y_log, X_log_const_2).fit()
print(loglog_2.summary())


                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.979
Model:                            OLS   Adj. R-squared:                  0.979
Method:                 Least Squares   F-statistic:                 1.855e+04
Date:                Mon, 05 Feb 2024   Prob (F-statistic):               0.00
Time:                        20:41:49   Log-Likelihood:                 5121.0
No. Observations:                6000   AIC:                        -1.021e+04
Df Residuals:                    5984   BIC:                        -1.010e+04
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           8.2667      0.008   1054.806      

In [84]:
results['prediction_ll_2'] = loglog_2.fittedvalues
results['residual_ll_2'] = loglog_2.resid

fig = px.scatter(
    results, x='prediction_ll_2', y='residual_ll_2', height = 350,
    labels={'prediction_ll_2':'Predicted values using the Log-Log Model and Interaction of color', 'residual_ll_2':'Residuals'}
)
fig.show()

In [85]:

X_train_log_2 = X_train.copy()
X_train_log_2['inter_E'] = X_train_log_2['Color_E'] * X_train_log_2['CaratWeight']
X_train_log_2['inter_F'] = X_train_log_2['Color_F'] * X_train_log_2['CaratWeight']
X_train_log_2['inter_G'] = X_train_log_2['Color_G'] * X_train_log_2['CaratWeight']
X_train_log_2['inter_H'] = X_train_log_2['Color_H'] * X_train_log_2['CaratWeight']
X_train_log_2['inter_I'] = X_train_log_2['Color_I'] * X_train_log_2['CaratWeight']
X_train_log_2['CaratWeight'] = X_train_log_2['CaratWeight'].transform(log)
X_train_log_const_2 = sm.add_constant(X_train_log_2)

X_test_log_2 = X_test.copy()

X_test_log_2['inter_E'] = X_test_log_2['Color_E'] * X_test_log_2['CaratWeight']
X_test_log_2['inter_F'] = X_test_log_2['Color_F'] * X_test_log_2['CaratWeight']
X_test_log_2['inter_G'] = X_test_log_2['Color_G'] * X_test_log_2['CaratWeight']
X_test_log_2['inter_H'] = X_test_log_2['Color_H'] * X_test_log_2['CaratWeight']
X_test_log_2['inter_I'] = X_test_log_2['Color_I'] * X_test_log_2['CaratWeight']
X_test_log_2['CaratWeight'] = X_test_log_2['CaratWeight'].transform(log)

X_test_log_const_2 = sm.add_constant(X_test_log_2)

loglog_2 = sm.OLS(Y_train_log, X_train_log_const_2).fit()
print(loglog_2.summary())

Y_pred_loglog_2 = np.exp(loglog_2.predict(X_test_log_const_2))
print(Y_pred_loglog_2.head(7))
percent_errors_2 = np.abs((Y_test['Price'] - Y_pred_loglog_2) / Y_test['Price']) *100
print("Log-Log Model 2 MAPE = ", np.mean(percent_errors_2), "%")

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.979
Model:                            OLS   Adj. R-squared:                  0.979
Method:                 Least Squares   F-statistic:                 1.485e+04
Date:                Mon, 05 Feb 2024   Prob (F-statistic):               0.00
Time:                        20:41:49   Log-Likelihood:                 4082.6
No. Observations:                4800   AIC:                            -8133.
Df Residuals:                    4784   BIC:                            -8030.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           8.2653      0.009    943.924      

## Model 3 (log-log and carat weight group dummies)

In [86]:
# create dummies for carat weight
from math import exp, log

bins = [0, 1, 1.5, 2, 1000] # Define the bins for cutting

labels = ['0-1', '1-1.5', '1.5-2', '>2'] # Define labels for each bin

X['CaratWeightgroup'] = pd.cut(X['CaratWeight'], bins=bins, labels=labels, right=False)
X = pd.get_dummies(data=X, columns = ['CaratWeightgroup'], drop_first=True)

X_log_3 = X.copy()

X_log_3.head()

,CaratWeight,Cut,Clarity,Polish,Symmetry,Color_E,Color_F,Color_G,Color_H,Color_I,CaratWeightgroup_1-1.5,CaratWeightgroup_1.5-2,CaratWeightgroup_>2
0,1.10,4,1,2,3,0,0,0,1,0,1,0,0
1,0.83,4,2,3,3,0,0,0,1,0,0,0,0
2,0.85,4,1,3,3,0,0,0,1,0,0,0,0
3,0.91,4,1,2,2,1,0,0,0,0,0,0,0
4,0.83,4,1,3,3,0,0,1,0,0,0,0,0


In [87]:
Y_log = Y['Price'].transform(log)
X_log_3['CaratWeight'] = X_log_3['CaratWeight'].transform(log)
X_log_3['inter_E'] = X_log_3['Color_E'] * X_log_3['CaratWeight']
X_log_3['inter_F'] = X_log_3['Color_F'] * X_log_3['CaratWeight']
X_log_3['inter_G'] = X_log_3['Color_G'] * X_log_3['CaratWeight']
X_log_3['inter_H'] = X_log_3['Color_H'] * X_log_3['CaratWeight']
X_log_3['inter_I'] = X_log_3['Color_I'] * X_log_3['CaratWeight']

X_log_3.head()

,CaratWeight,Cut,Clarity,Polish,Symmetry,Color_E,Color_F,Color_G,Color_H,Color_I,CaratWeightgroup_1-1.5,CaratWeightgroup_1.5-2,CaratWeightgroup_>2,inter_E,inter_F,inter_G,inter_H,inter_I
0,0.095310,4,1,2,3,0,0,0,1,0,1,0,0,0.000000,0.0,0.00000,0.095310,0.0
1,-0.186330,4,2,3,3,0,0,0,1,0,0,0,0,-0.000000,-0.0,-0.00000,-0.186330,-0.0
2,-0.162519,4,1,3,3,0,0,0,1,0,0,0,0,-0.000000,-0.0,-0.00000,-0.162519,-0.0
3,-0.094311,4,1,2,2,1,0,0,0,0,0,0,0,-0.094311,-0.0,-0.00000,-0.000000,-0.0
4,-0.186330,4,1,3,3,0,0,1,0,0,0,0,0,-0.000000,-0.0,-0.18633,-0.000000,-0.0


In [88]:
#MODEL3 SUMMARY
X_log_const_3 = sm.add_constant(X_log_3)
loglog_3 = sm.OLS(Y_log, X_log_const_3).fit()
print(loglog_3.summary())


                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.981
Model:                            OLS   Adj. R-squared:                  0.981
Method:                 Least Squares   F-statistic:                 1.754e+04
Date:                Mon, 05 Feb 2024   Prob (F-statistic):               0.00
Time:                        20:41:50   Log-Likelihood:                 5493.8
No. Observations:                6000   AIC:                        -1.095e+04
Df Residuals:                    5981   BIC:                        -1.082e+04
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                      8

In [89]:
#Plot the residual

results = pd.DataFrame()
results['Price'] = df['Price']

results['prediction_ll_3'] = loglog_3.fittedvalues
results['residual_ll_3'] = loglog_3.resid

fig = px.scatter(
    results, x='prediction_ll_3', y='residual_ll_3', height = 350,
    labels={'prediction_ll_3':'Predicted values using the Log-Log Model and Interaction of color', 'residual_ll_3':'Residuals'}
)
fig.show()

In [90]:
# Cross-validation

from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

#Add a constant to the X's:
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)
Y_train_log = Y_train['Price'].transform(log)

X_train_const.head()

,const,CaratWeight,Cut,Clarity,Polish,Symmetry,Color_E,Color_F,Color_G,Color_H,Color_I,CaratWeightgroup_1-1.5,CaratWeightgroup_1.5-2,CaratWeightgroup_>2
3897,1.0,1.34,3,2,2,1,0,0,0,1,0,1,0,0
5628,1.0,2.57,4,2,3,2,0,0,0,1,0,0,0,1
1756,1.0,1.01,4,4,3,3,0,0,0,0,0,1,0,0
2346,1.0,2.09,4,2,2,2,0,1,0,0,0,0,0,1
2996,1.0,0.90,3,1,2,2,0,0,0,0,1,0,0,0


In [91]:
X_train_log_3 = X_train.copy()

X_train_log_3['inter_E'] = X_train_log_3['Color_E'] * X_train_log_3['CaratWeight']
X_train_log_3['inter_F'] = X_train_log_3['Color_F'] * X_train_log_3['CaratWeight']
X_train_log_3['inter_G'] = X_train_log_3['Color_G'] * X_train_log_3['CaratWeight']
X_train_log_3['inter_H'] = X_train_log_3['Color_H'] * X_train_log_3['CaratWeight']
X_train_log_3['inter_I'] = X_train_log_3['Color_I'] * X_train_log_3['CaratWeight']
X_train_log_const_3 = sm.add_constant(X_train_log_3)

X_test_log_3 = X_test.copy()

X_test_log_3['inter_E'] = X_test_log_3['Color_E'] * X_test_log_3['CaratWeight']
X_test_log_3['inter_F'] = X_test_log_3['Color_F'] * X_test_log_3['CaratWeight']
X_test_log_3['inter_G'] = X_test_log_3['Color_G'] * X_test_log_3['CaratWeight']
X_test_log_3['inter_H'] = X_test_log_3['Color_H'] * X_test_log_3['CaratWeight']
X_test_log_3['inter_I'] = X_test_log_3['Color_I'] * X_test_log_3['CaratWeight']

X_test_log_const_3 = sm.add_constant(X_test_log_3)

loglog_3 = sm.OLS(Y_train_log, X_train_log_const_3).fit()
print(loglog_3.summary())
# print(Y_pred_loglog_3.head(7))
Y_pred_loglog_3 = np.exp(loglog_3.predict(X_test_log_const_3))
percent_errors_3 = np.abs((Y_test['Price'] - Y_pred_loglog_3) / Y_test['Price']) *100
print("Log-Log Model 1 MAPE = ", np.mean(percent_errors_3), "%")

                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.975
Model:                            OLS   Adj. R-squared:                  0.975
Method:                 Least Squares   F-statistic:                 1.052e+04
Date:                Mon, 05 Feb 2024   Prob (F-statistic):               0.00
Time:                        20:41:50   Log-Likelihood:                 3702.6
No. Observations:                4800   AIC:                            -7367.
Df Residuals:                    4781   BIC:                            -7244.
Df Model:                          18                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                      6

##Model 4: Log-Log Model with interaction variable of Carat Weight * Cut

In [92]:
# Model 4: Log-Log Model with Interaction Term (CaratWeight * Cut)
from math import log

# Create interaction term
df['CaratWeight_Cut_interaction'] = df['CaratWeight'] * df['Cut']

# Transform the dependent variable
Y_log = Y['Price'].transform(log)

# Transform the independent variables
X_log_interaction = X.copy()
X_log_interaction['CaratWeight_Cut_interaction'] = df['CaratWeight_Cut_interaction'].transform(log)

# Add a constant to the independent variables
X_log_interaction_const = sm.add_constant(X_log_interaction)

# Fit the log-log regression model with the interaction term
loglog_interaction_model = sm.OLS(Y_log, X_log_interaction_const).fit()
print(loglog_interaction_model.summary())

# Insert code for visualization
results['prediction_llm_interaction'] = loglog_interaction_model.fittedvalues
results['residual_llm_interaction'] = loglog_interaction_model.resid

fig = px.scatter(
    results, x='prediction_llm_interaction', y='residual_llm_interaction', height=350,
    labels={'prediction_llm_interaction': 'Predicted values using the Log-Log Model with Interaction Term', 'residual_llm_interaction': 'Residuals'}
)
fig.show()

# Transform the predicted values back to the original scale
Y_pred_interaction = np.exp(loglog_interaction_model.predict(X_log_interaction_const))

# Calculate the absolute percentage error for each prediction
absolute_percentage_errors = np.abs((np.exp(Y_log) - Y_pred_interaction) / np.exp(Y_log)) * 100

# Calculate Mean Absolute Percentage Error (MAPE)
mape_interaction = np.mean(absolute_percentage_errors)
print("Log-Log Model with Interaction Term MAPE =", mape_interaction, "%")


                            OLS Regression Results                            
Dep. Variable:                  Price   R-squared:                       0.973
Model:                            OLS   Adj. R-squared:                  0.973
Method:                 Least Squares   F-statistic:                 1.568e+04
Date:                Mon, 05 Feb 2024   Prob (F-statistic):               0.00
Time:                        20:41:50   Log-Likelihood:                 4425.1
No. Observations:                6000   AIC:                            -8820.
Df Residuals:                    5985   BIC:                            -8720.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const             

Log-Log Model with Interaction Term MAPE = 9.006161845463874 %


##Model 5: Log Linear Model on three Parts of the data

In [ ]:
df = pd.read_excel("UV6248-XLS-ENG.xls", sheet_name = "Raw Data", skiprows=2)
df = df.rename(columns={'Carat Weight': 'CaratWeight'})

# take out the final test data (i.e. with price value as NaN)
final_test_data = df[df['Price'].isna()]

# drop the values from the final test data from original df
df=df.dropna()

# Handle the Low values
df['Clarity'] = df['Clarity'].replace({'IF':'IF_FL','FL':'IF_FL'})
print(df.Clarity.value_counts())

df_1=df[df['CaratWeight']<1]

# explore data with log price and log caratweight
from math import log
df_log = df_1.copy()
df_log['Price']=df_log['Price'].transform(log)
# df_log['CaratWeight']=df_log['CaratWeight'].transform(log)


df_log = pd.get_dummies(data=df_log, columns = ['Color', 'Report'], drop_first=True)
scale_map_cut = {'Fair':1, 'Good':2, 'Very Good':3, 'Ideal':4,  'Signature-Ideal':5}
scale_map_clarity = {'SI1':1, 'VVS2':3, 'VVS1':3, 'VS2':2, 'VS1':2, 'IF_FL':4}
scale_map_polish = {'G':1, 'VG':2, 'EX':3, 'ID':3}

df_log["Cut"] = df_log["Cut"].replace(scale_map_cut)
df_log["Clarity"] = df_log["Clarity"].replace(scale_map_clarity)
df_log["Polish"] = df_log["Polish"].replace(scale_map_polish)
df_log["Symmetry"] = df_log["Symmetry"].replace(scale_map_polish)


from statsmodels.stats.outliers_influence import variance_inflation_factor
# VIF dataframe

vif_data = pd.DataFrame()
vif_data["feature"] = df_log.drop(['Price','ID'], axis=1).columns


# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(df_log.drop(['Price','ID'], axis=1).values, i)
                          for i in range(len(df_log.drop(['Price','ID'], axis=1).columns))]
print("Previous VIF: \n", vif_data)

# mean centering approach for variables with high vif value
df_log['CaratWeight']=df_log['CaratWeight']-df_log['CaratWeight'].mean()
df_log['Cut']=df_log['Cut']-df_log['Cut'].mean()
df_log['Polish']=df_log['Polish']-df_log['Polish'].mean()
df_log['Symmetry']=df_log['Symmetry']-df_log['Symmetry'].mean()
vif_data = pd.DataFrame()
vif_data["feature"] = df_log.drop(['Price','ID'], axis=1).columns


# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(df_log.drop(['Price','ID'], axis=1).values, i)
                          for i in range(len(df_log.drop(['Price','ID'], axis=1).columns))]
print("\n \n VIF after data transformation: \n",vif_data)

# Adding interaction terms
for color in ['E', 'F', 'G', 'H', 'I']:
    interaction_term = f'CaratWeight_color_{color}'
    df_log[interaction_term] = df_log['CaratWeight'] * df_log[f'Color_{color}']

df_log=df_log.rename(columns={'CaratWeight': 'CaratWeight_MC'})


# Dependent Variable
Y = df_log[(['Price'])]
Y.head()

# Independent Variables
X = df_log.drop(['Price','ID','Report_GIA','Symmetry'], axis=1)
X.dtypes

# Building LM Model

# In this package, by default, the regression will have no intercept, hence we need to manually add it to the X matrix, and call the result X_const
X_const = sm.add_constant(X)
X_const.head()

# # Fit a linear regression model with vector Y as dependent and matrix X_sm as independent
lm = sm.OLS(Y, X_const).fit()

# # # Display the summary of model results
print(lm.summary())

# Residual Plot
# Compute the residuals
results = pd.DataFrame()
results['Price'] = df_log['Price']
results['prediction_lm'] = lm.fittedvalues
results['residual_lm'] = lm.resid

fig = px.scatter(
    results, x='prediction_lm', y='residual_lm', height = 350,
    labels={'prediction_lm':'Predicted values using the Log Log Model', 'residual_lm':'Residuals using the Log Log Model'}
)
fig.show()

# Cross Validation
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

#Add a constant to the X's:
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

# Fit a linear regression model to the training data
lm = sm.OLS(Y_train, X_train_const).fit()

# Use the trained model to predict the prices for the testing data. Call the vector of predicted prices Y_pred
Y_pred = lm.predict(X_test_const)
Y_pred_exp = np.exp(Y_pred)
percent_errors = np.abs((np.exp(Y_test['Price']) - Y_pred_exp) / np.exp(Y_test['Price'])) *100
print("Log Linear Model MAPE = ", np.mean(percent_errors), "%")

In [ ]:
df = pd.read_excel("UV6248-XLS-ENG.xls", sheet_name = "Raw Data", skiprows=2)
df = df.rename(columns={'Carat Weight': 'CaratWeight'})

# take out the final test data (i.e. with price value as NaN)
final_test_data = df[df['Price'].isna()]

# drop the values from the final test data from original df
df=df.dropna()

# Handle the Low values
df['Clarity'] = df['Clarity'].replace({'IF':'IF_FL','FL':'IF_FL'})
print(df.Clarity.value_counts())

df_2 = df[(df['CaratWeight'] >= 1) & (df['CaratWeight'] < 2)]

# explore data with log price and log caratweight
from math import log
df_log = df_2.copy()
df_log['Price']=df_log['Price'].transform(log)
df_log['CaratWeight']=df_log['CaratWeight'].transform(log)


df_log = pd.get_dummies(data=df_log, columns = ['Color', 'Report'], drop_first=True)
scale_map_cut = {'Fair':1, 'Good':2, 'Very Good':3, 'Ideal':4,  'Signature-Ideal':5}
scale_map_clarity = {'SI1':1, 'VVS2':3, 'VVS1':3, 'VS2':2, 'VS1':2, 'IF_FL':4}
scale_map_polish = {'G':1, 'VG':2, 'EX':3, 'ID':3}

df_log["Cut"] = df_log["Cut"].replace(scale_map_cut)
df_log["Clarity"] = df_log["Clarity"].replace(scale_map_clarity)
df_log["Polish"] = df_log["Polish"].replace(scale_map_polish)
df_log["Symmetry"] = df_log["Symmetry"].replace(scale_map_polish)


from statsmodels.stats.outliers_influence import variance_inflation_factor
# VIF dataframe

vif_data = pd.DataFrame()
vif_data["feature"] = df_log.drop(['Price','ID'], axis=1).columns


# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(df_log.drop(['Price','ID'], axis=1).values, i)
                          for i in range(len(df_log.drop(['Price','ID'], axis=1).columns))]
print("Previous VIF: \n", vif_data)

# mean centering approach for variables with high vif value
df_log['CaratWeight']=df_log['CaratWeight']-df_log['CaratWeight'].mean()
df_log['Cut']=df_log['Cut']-df_log['Cut'].mean()
df_log['Polish']=df_log['Polish']-df_log['Polish'].mean()
df_log['Symmetry']=df_log['Symmetry']-df_log['Symmetry'].mean()
vif_data = pd.DataFrame()
vif_data["feature"] = df_log.drop(['Price','ID'], axis=1).columns


# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(df_log.drop(['Price','ID'], axis=1).values, i)
                          for i in range(len(df_log.drop(['Price','ID'], axis=1).columns))]
print("\n \n VIF after data transformation: \n",vif_data)

# Adding interaction terms
for color in ['E', 'F', 'G', 'H', 'I']:
    interaction_term = f'CaratWeight_color_{color}'
    df_log[interaction_term] = df_log['CaratWeight'] * df_log[f'Color_{color}']

df_log=df_log.rename(columns={'CaratWeight': 'CaratWeight_MC'})


# Dependent Variable
Y = df_log[(['Price'])]
Y.head()

# Independent Variables
X = df_log.drop(['Price','ID','Report_GIA'], axis=1)
X.dtypes

# Building LM Model

# In this package, by default, the regression will have no intercept, hence we need to manually add it to the X matrix, and call the result X_const
X_const = sm.add_constant(X)
X_const.head()

# # Fit a linear regression model with vector Y as dependent and matrix X_sm as independent
lm = sm.OLS(Y, X_const).fit()

# # # Display the summary of model results
print(lm.summary())

# Residual Plot
# Compute the residuals
results = pd.DataFrame()
results['Price'] = df_log['Price']
results['prediction_lm'] = lm.fittedvalues
results['residual_lm'] = lm.resid

fig = px.scatter(
    results, x='prediction_lm', y='residual_lm', height = 350,
    labels={'prediction_lm':'Predicted values using the Log Log Model', 'residual_lm':'Residuals using the Log Log Model'}
)
fig.show()

# Cross Validation
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

#Add a constant to the X's:
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

# Fit a linear regression model to the training data
lm = sm.OLS(Y_train, X_train_const).fit()

# Use the trained model to predict the prices for the testing data. Call the vector of predicted prices Y_pred
Y_pred = lm.predict(X_test_const)
Y_pred_exp = np.exp(Y_pred)
percent_errors = np.abs((np.exp(Y_test['Price']) - Y_pred_exp) / np.exp(Y_test['Price'])) *100
print("Log Linear Model MAPE = ", np.mean(percent_errors), "%")

In [ ]:
df = pd.read_excel("UV6248-XLS-ENG.xls", sheet_name = "Raw Data", skiprows=2)
df = df.rename(columns={'Carat Weight': 'CaratWeight'})

# take out the final test data (i.e. with price value as NaN)
final_test_data = df[df['Price'].isna()]

# drop the values from the final test data from original df
df=df.dropna()

# Handle the Low values
df['Clarity'] = df['Clarity'].replace({'IF':'IF_FL','FL':'IF_FL'})
print(df.Clarity.value_counts())

df_3 = df[df['CaratWeight'] >= 2]

# explore data with log price and log caratweight
from math import log
df_log = df_3.copy()
df_log['Price']=df_log['Price'].transform(log)
df_log['CaratWeight']=df_log['CaratWeight'].transform(log)


df_log = pd.get_dummies(data=df_log, columns = ['Color', 'Report'], drop_first=True)
scale_map_cut = {'Fair':1, 'Good':2, 'Very Good':3, 'Ideal':4,  'Signature-Ideal':5}
scale_map_clarity = {'SI1':1, 'VVS2':3, 'VVS1':3, 'VS2':2, 'VS1':2, 'IF_FL':4}
scale_map_polish = {'G':1, 'VG':2, 'EX':3, 'ID':3}

df_log["Cut"] = df_log["Cut"].replace(scale_map_cut)
df_log["Clarity"] = df_log["Clarity"].replace(scale_map_clarity)
df_log["Polish"] = df_log["Polish"].replace(scale_map_polish)
df_log["Symmetry"] = df_log["Symmetry"].replace(scale_map_polish)


from statsmodels.stats.outliers_influence import variance_inflation_factor
# VIF dataframe

vif_data = pd.DataFrame()
vif_data["feature"] = df_log.drop(['Price','ID'], axis=1).columns


# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(df_log.drop(['Price','ID'], axis=1).values, i)
                          for i in range(len(df_log.drop(['Price','ID'], axis=1).columns))]
print("Previous VIF: \n", vif_data)

# mean centering approach for variables with high vif value
df_log['CaratWeight']=df_log['CaratWeight']-df_log['CaratWeight'].mean()
df_log['Cut']=df_log['Cut']-df_log['Cut'].mean()
df_log['Polish']=df_log['Polish']-df_log['Polish'].mean()
df_log['Symmetry']=df_log['Symmetry']-df_log['Symmetry'].mean()
vif_data = pd.DataFrame()
vif_data["feature"] = df_log.drop(['Price','ID'], axis=1).columns


# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(df_log.drop(['Price','ID'], axis=1).values, i)
                          for i in range(len(df_log.drop(['Price','ID'], axis=1).columns))]
print("\n \n VIF after data transformation: \n",vif_data)

# Adding interaction terms
for color in ['E', 'F', 'G', 'H', 'I']:
    interaction_term = f'CaratWeight_color_{color}'
    df_log[interaction_term] = df_log['CaratWeight'] * df_log[f'Color_{color}']

df_log=df_log.rename(columns={'CaratWeight': 'CaratWeight_MC'})


# Dependent Variable
Y = df_log[(['Price'])]
Y.head()

# Independent Variables
X = df_log.drop(['Price','ID','Report_GIA','Symmetry'], axis=1)
X.dtypes

# Building LM Model

# In this package, by default, the regression will have no intercept, hence we need to manually add it to the X matrix, and call the result X_const
X_const = sm.add_constant(X)
X_const.head()

# # Fit a linear regression model with vector Y as dependent and matrix X_sm as independent
lm = sm.OLS(Y, X_const).fit()

# # # Display the summary of model results
print(lm.summary())

# Residual Plot
# Compute the residuals
results = pd.DataFrame()
results['Price'] = df_log['Price']
results['prediction_lm'] = lm.fittedvalues
results['residual_lm'] = lm.resid

fig = px.scatter(
    results, x='prediction_lm', y='residual_lm', height = 350,
    labels={'prediction_lm':'Predicted values using the Log Log Model', 'residual_lm':'Residuals using the Log Log Model'}
)
fig.show()

# Cross Validation
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

#Add a constant to the X's:
X_train_const = sm.add_constant(X_train)
X_test_const = sm.add_constant(X_test)

# Fit a linear regression model to the training data
lm = sm.OLS(Y_train, X_train_const).fit()

# Use the trained model to predict the prices for the testing data. Call the vector of predicted prices Y_pred
Y_pred = lm.predict(X_test_const)
Y_pred_exp = np.exp(Y_pred)
percent_errors = np.abs((np.exp(Y_test['Price']) - Y_pred_exp) / np.exp(Y_test['Price'])) *100
print("Log Linear Model MAPE = ", np.mean(percent_errors), "%")

# Produce and export the final result

After choosing the first model (Log-Log Model with price and carat weight both logged, MAPE = 7.86%), we will forecast the missing prices in the data set by the information provided.



In [ ]:
def load_data(file_path: str, sheet_name: str, skip_rows: int) -> pd.DataFrame:
    """
    Load data from an Excel file.

    Parameters:
    - file_path (str): Path to the Excel file.
    - sheet_name (str): Name of the sheet containing the data.
    - skip_rows (int): Number of rows to skip at the beginning.

    Returns:
    - pd.DataFrame: Loaded DataFrame.
    """
    df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=skip_rows)
    df = df.rename(columns={'Carat Weight': 'CaratWeight'})
    return df

def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocess the data by handling missing values and transforming features.

    Parameters:
    - df (pd.DataFrame): Original DataFrame.

    Returns:
    - pd.DataFrame: Preprocessed DataFrame.
    """

    df['Clarity'] = df['Clarity'].replace({'IF': 'IF_FL', 'FL': 'IF_FL'})

    df_log = df.copy()
    df_log['Price'] = df_log['Price'].transform(np.log)
    df_log['CaratWeight'] = df_log['CaratWeight'].transform(np.log)

    df_log = pd.get_dummies(data=df_log, columns=['Color', 'Report'], drop_first=True)

    scale_map_cut = {'Fair': 1, 'Good': 2, 'Very Good': 3, 'Ideal': 4, 'Signature-Ideal': 5}
    scale_map_clarity = {'SI1': 1, 'VVS2': 3, 'VVS1': 3, 'VS2': 2, 'VS1': 2, 'IF_FL': 4}
    scale_map_polish = {'G': 1, 'VG': 2, 'EX': 3, 'ID': 3}

    df_log["Cut"] = df_log["Cut"].replace(scale_map_cut)
    df_log["Clarity"] = df_log["Clarity"].replace(scale_map_clarity)
    df_log["Polish"] = df_log["Polish"].replace(scale_map_polish)
    df_log["Symmetry"] = df_log["Symmetry"].replace(scale_map_polish)

    vif_data = pd.DataFrame()
    vif_data["feature"] = df_log.drop(['Price', 'ID'], axis=1).columns

    vif_data["VIF"] = [variance_inflation_factor(
        df_log.drop(['Price', 'ID'], axis=1).values, i)
        for i in range(len(df_log.drop(['Price', 'ID'], axis=1).columns))]
    print("Previous VIF: \n", vif_data)

    df_log['CaratWeight'] = df_log['CaratWeight'] - df_log['CaratWeight'].mean()
    df_log['Cut'] = df_log['Cut'] - df_log['Cut'].mean()
    df_log['Polish'] = df_log['Polish'] - df_log['Polish'].mean()
    df_log['Symmetry'] = df_log['Symmetry'] - df_log['Symmetry'].mean()

    vif_data = pd.DataFrame()
    vif_data["feature"] = df_log.drop(['Price', 'ID'], axis=1).columns

    vif_data["VIF"] = [variance_inflation_factor(
        df_log.drop(['Price', 'ID'], axis=1).values, i)
        for i in range(len(df_log.drop(['Price', 'ID'], axis=1).columns))]
    print("\n \n VIF after data transformation: \n", vif_data)

    for color in ['E', 'F', 'G', 'H', 'I']:
        interaction_term = f'CaratWeight_color_{color}'
        df_log[interaction_term] = df_log['CaratWeight'] * df_log[f'Color_{color}']

    # df_log = df_log.rename(columns={'CaratWeight': 'CaratWeight_MC'})
    return df_log

def build_lm_model(X: pd.DataFrame, Y: pd.DataFrame) -> sm.regression.linear_model.RegressionResults:
    """
    Build a linear regression model.

    Parameters:
    - X (pd.DataFrame): Independent variables.
    - Y (pd.DataFrame): Dependent variable.

    Returns:
    - sm.regression.linear_model.RegressionResults: Linear regression model results.
    """
    X_const = sm.add_constant(X)
    lm = sm.OLS(Y, X_const).fit()
    return lm

def visualize_residual_plot(results: pd.DataFrame) -> None:
    """
    Visualize a residual plot.

    Parameters

:
    - results (pd.DataFrame): DataFrame containing model results and residuals.
    """
    fig = px.scatter(
        results, x='prediction_lm', y='residual_lm', height=350,
        labels={'prediction_lm': 'Predicted values using the Log Log Model', 'residual_lm': 'Residuals using the Log Log Model'}
    )
    fig.show()

def perform_cross_validation(X_train: pd.DataFrame, X_test: pd.DataFrame,
                              Y_train: pd.DataFrame, Y_test: pd.DataFrame) -> None:
    """
    Perform cross-validation.

    Parameters:
    - X_train (pd.DataFrame): Training set independent variables.
    - X_test (pd.DataFrame): Testing set independent variables.
    - Y_train (pd.DataFrame): Training set dependent variable.
    - Y_test (pd.DataFrame): Testing set dependent variable.
    """
    X_train_const = sm.add_constant(X_train)
    X_test_const = sm.add_constant(X_test)

    lm = sm.OLS(Y_train, X_train_const).fit()

    Y_pred = lm.predict(X_test_const)
    Y_pred_exp = np.exp(Y_pred)
    percent_errors = np.abs((np.exp(Y_test['Price']) - Y_pred_exp) / np.exp(Y_test['Price'])) * 100
    print("Log Log Model MAPE = ", np.mean(percent_errors), "%")

# Load data
file_path = "UV6248-XLS-ENG.xls"
sheet_name = "Raw Data"
skip_rows = 2
df = load_data(file_path, sheet_name, skip_rows)
final_test_data = df
df_dropped = df.dropna()

# Preprocess data
df_log = preprocess_data(df_dropped)
final_test_data_log=preprocess_data(final_test_data)


# Build LM Model
Y = df_log[(['Price'])]
X = df_log.drop(['Price', 'ID', 'Report_GIA'], axis=1)
lm_model = build_lm_model(X, Y)

# Display the summary of the model
print(lm_model.summary())

# Residual Plot
results_df = pd.DataFrame()
results_df['Price'] = df_log['Price']
results_df['prediction_lm'] = lm_model.fittedvalues
results_df['residual_lm'] = lm_model.resid

# Visualize Residual Plot
visualize_residual_plot(results_df)

# Cross Validation
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

perform_cross_validation(X_train, X_test, Y_train, Y_test)


# Use the trained model to predict the prices for the testing data. Call the vector of predicted prices Y_pred
X_test = final_test_data_log.drop(['Price','ID','Report_GIA'], axis=1)
X_test_const = sm.add_constant(X_test)
Y_pred = lm_model.predict(X_test_const)
Y_pred_exp = np.exp(Y_pred)
final_test_data['Predicted_Price']= Y_pred_exp
print('Predictions: \n\n',final_test_data)
final_test_data.to_csv("Sarah_gets_diamond_model_output.csv")